# Energy Meter Examples

## Linux Kernel HWMon

More details can be found at https://github.com/ARM-software/lisa/wiki/Energy-Meters-Requirements#linux-hwmon.

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()

2016-12-08 12:40:25,090 INFO    : root         : Using LISA logging configuration:
2016-12-08 12:40:25,091 INFO    : root         :   /home/vagrant/lisa/logging.conf


#### Import required modules

In [2]:
# Generate plots inline
%matplotlib inline

import os

# Support to access the remote target
import devlib
from env import TestEnv

# RTApp configurator for generation of PERIODIC tasks
from wlgen import RTA, Ramp

## Target Configuration
The target configuration is used to describe and configure your test environment.
You can find more details in **examples/utils/testenv_example.ipynb**.

In [3]:
# Setup target configuration
my_conf = {

    # Target platform and board
    "platform"    : 'linux',
    "board"       : 'juno',
    "host"        : '192.168.0.1',

    # Devlib modules to load
    "modules" : ["cpufreq"], # Required by rt-app calibration
    
    # Folder where all the results will be collected
    "results_dir" : "EnergyMeter_HWMON",

    # Energy Meters Configuration for BayLibre's ACME Cape
    "emeter" : {
        "instrument" : "hwmon",
        "conf" : {
            # Prefixes of the HWMon labels
            'sites' : ['a53', 'a57'],
            # Type of hardware monitor to be used
            'kinds' : ['energy']
        },
        'channel_map' : {
            'LITTLE' : 'a53',
            'big' : 'a57',
        }
    },
    
    # Tools required by the experiments
    "tools"   : [ 'trace-cmd', 'rt-app' ],
    
    # Comment this line to calibrate RTApp in your own platform
    # "rtapp-calib" :  {"0": 360, "1": 142, "2": 138, "3": 352, "4": 352, "5": 353},
}

In [4]:
# Initialize a test environment using:
te = TestEnv(my_conf, wipe=False, force_new=True)
target = te.target

03:59:42  INFO    :         Target - Using base path: /data/lisa
03:59:42  INFO    :         Target - Loading custom (inline) target configuration
03:59:42  INFO    :         Target - Devlib modules to load: ['bl', 'hwmon', 'cpufreq']
03:59:42  INFO    :         Target - Connecting linux target:
03:59:42  INFO    :         Target -   username : root
03:59:42  INFO    :         Target -       host : 192.168.0.1
03:59:42  INFO    :         Target -   password : 
03:59:42  INFO    :         Target - Connection settings:
03:59:42  INFO    :         Target -    {'username': 'root', 'host': '192.168.0.1', 'password': ''}
04:00:23  INFO    :         Target - Initializing target workdir:
04:00:23  INFO    :         Target -    /root/devlib-target
04:00:28  INFO    :         Target - Topology:
04:00:28  INFO    :         Target -    [[0, 3, 4, 5], [1, 2]]
04:00:30  INFO    :       Platform - Loading default EM:
04:00:30  INFO    :       Platform -    /data/lisa/libs/utils/platforms/juno.json
04

## Workload Execution and Power Consumptions Samping

Detailed information on RTApp can be found in **examples/wlgen/rtapp_example.ipynb**.

Each **EnergyMeter** derived class has two main methods: **reset** and **report**.
 - The **reset** method will reset the energy meter and start sampling from channels specified in the target configuration. <br>
 - The **report** method will stop capture and will retrieve the energy consumption data. This returns an EnergyReport composed of the measured channels energy and the report file. Each of the samples can also be obtained, as you can see below.

In [5]:
# Create and RTApp RAMP task
rtapp = RTA(te.target, 'ramp', calibration=te.calibration())
rtapp.conf(kind='profile',
           params={
                'ramp' : Ramp(
                    start_pct =  60,
                    end_pct   =  20,
                    delta_pct =   5,
                    time_s    =   0.5).get()
          })

# EnergyMeter Start
te.emeter.reset()

rtapp.run(out_dir=te.res_dir)

# EnergyMeter Stop and samples collection
nrg_report = te.emeter.report(te.res_dir)

04:00:30  INFO    :          WlGen - Setup new workload ramp
04:00:30  INFO    :          RTApp - Workload duration defined by longest task
04:00:30  INFO    :          RTApp - Default policy: SCHED_OTHER
04:00:30  INFO    :          RTApp - ------------------------
04:00:30  INFO    :          RTApp - task [ramp], sched: using default policy
04:00:30  INFO    :          RTApp -  | calibration CPU: 1
04:00:30  INFO    :          RTApp -  | loops count: 1
04:00:30  INFO    :          RTApp - + phase_000001: duration 0.500000 [s] (5 loops)
04:00:30  INFO    :          RTApp - |  period   100000 [us], duty_cycle  60 %
04:00:30  INFO    :          RTApp - |  run_time  60000 [us], sleep_time  40000 [us]
04:00:30  INFO    :          RTApp - + phase_000002: duration 0.500000 [s] (5 loops)
04:00:30  INFO    :          RTApp - |  period   100000 [us], duty_cycle  55 %
04:00:30  INFO    :          RTApp - |  run_time  55000 [us], sleep_time  45000 [us]
04:00:30  INFO    :          RTApp - + phas

In [6]:
logging.info("Collected data:")
!tree $te.res_dir

04:00:41  INFO    : Collected data:


/data/lisa/results/EnergyMeter_HWMON
├── energy.json
├── output.log
├── ramp_00.json
├── rt-app-ramp-0.log
├── trace.dat
├── trace.raw.txt
└── trace.txt

0 directories, 7 files


## Power Measurements Data

In [7]:
logging.info("Measured channels energy:")
logging.info("%s", nrg_report.channels)

04:00:41  INFO    : Measured channels energy:
04:00:41  INFO    : {'big': 14.410717999999179, 'LITTLE': 1.1058980000016163}


In [8]:
logging.info("Generated energy file:")
logging.info("  %s", nrg_report.report_file)
!cat $nrg_report.report_file

04:00:41  INFO    : Generated energy file:
04:00:41  INFO    :   /data/lisa/results/EnergyMeter_HWMON/energy.json


{
    "LITTLE": 1.1058980000016163, 
    "big": 14.410717999999179
}